In [12]:
import os
import openai
from app import my_prompt

AI_API_KEY = os.getenv('AI_API_KEY') or None

if AI_API_KEY is None:
    raise BaseException('Missing sec configuration (video_summarizer)')

openai.api_key = AI_API_KEY

In [2]:
with open('temp_data/temp_video_subtitles.txt', 'r') as f:
    transcript = f.read()

In [19]:
import logging
import backoff
import openai
import time

from openai.error import RateLimitError

openai.api_key = AI_API_KEY

logging.getLogger('backoff').addHandler(logging.StreamHandler())
# logging.getLogger('backoff').setLevel(logging.ERROR)

@backoff.on_exception(backoff.constant, openai.error.RateLimitError, interval=5)
def completions_with_backoff(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
            model='gpt-4',
            messages=messages,
            temperature=0
        )
    
    return response.choices[0].message["content"]


# Define a function that adds a delay to a Completion API call
def delayed_completion(prompt, delay_in_seconds: float = 1, retries: int = 10):
    """Delay a completion by a specified amount of time."""

    # Call the Completion API and return the result
    messages = [{"role": "user", "content": prompt}]
    response = None
    retry_attempts = 1
    while response is None:
        try:
            response = openai.ChatCompletion.create(
                    model='gpt-4',
                    messages=messages,
                    temperature=0
                )
        except RateLimitError as rate_err:
            print('Sleeping for {}s, retry attempt #{}, error_msg: {}'.format( delay_in_seconds, retry_attempts, rate_err.user_message))
            time.sleep( delay_in_seconds )
            response = None
            retry_attempts = retry_attempts + 1
            if retry_attempts >= retries:
                raise BaseException('Exceeded number of retries [{}]'.format( retries ))
    
    return response.choices[0].message["content"]

In [21]:
words = transcript.split(' ')

MAX_WORDS = 750*7-len( my_prompt.BATCH_PROMPT_CUSTOM_UNCLE_BRUCE.split(' ') )
DELAY_IN_SECONDS = 7

# enc = tiktoken.encoding_for_model( 'gpt-4' )
# num_tokens = len(enc.encode(prompt))
f_out = open('test_delete.txt', 'w')
batches = []
for i in range(0, len(words), MAX_WORDS):
    text_snippet = ' '.join(words[i:i+MAX_WORDS])
    single_batch = '{}\n---\n{}'.format( my_prompt.BATCH_PROMPT_CUSTOM_UNCLE_BRUCE, text_snippet)
    batches.append( single_batch )
    print ('{}:{}\n'.format(i, single_batch))
    f_out.write( '\n{}:{}\n'.format(i, single_batch) )
f_out.close()

# delete file
with open('temp_response.txt', 'w') as f_out:
     f_out.write('')
for i,single_batch in enumerate(batches):     
        # response = completions_with_backoff( single_batch )
        response = delayed_completion( single_batch, DELAY_IN_SECONDS)
        print( '#{}: {}'.format(i, response) )
        with open('temp_response.txt', 'a') as f_out:
            f_out.write( '- {}\n'.format(response) )
    
# response = get_completion(batch)
# f_out.close()


0:You are the stock analyst. I will give you a YouTube video transcript, a text delimited by triple backticks.The transcript contains daily updates on the stock market, advice on option trading strategies, and personal life stories. You should summarise the text focusing on stocks.
---
Okay. Okay, everybody. How you doing? It's Friday. It's Friday for a lot of you. I know that's what you live for. Congratulations. It's September the 29th, Friday, option expiry day 2023 coming to you live from Vienna, Austria. Jennifer and I welcome all of you to the to the broadcast. Glad you're here. Just so you those of you don't know. This is what the neighborhood looks like where we are right now. This is kind of what it looks like. These have these buildings around us are sort of four or five, six stories tall. Old downtown Vienna. Now, to be fair, a lot of these buildings were built many years ago, but they've they've gone through renovations over the years. Pretty well. I mean, every single busi

InvalidRequestError: This model's maximum context length is 8192 tokens. However, your messages resulted in 8362 tokens. Please reduce the length of the messages.

In [ ]:
for i in range(0, len(words), MAX_WORDS):
    print (i)
messages = [{"role": "user", "content": prompt}]
response = openai.ChatCompletion.create(
        model='gpt-4',
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output,
        # engine=config_details['MS_CHAT_GPT_MODEL']
    )

print (response)

In [4]:
import tiktoken 

with open('temp_data/temp_summary_batches_responses.txt', 'r') as f_in:
    summaries = f_in.read()

print( len( summaries ))

enc = tiktoken.encoding_for_model('gpt-4')
words = enc.encode( summaries )

print( len(words))

7248
1448


In [13]:
import openai

prompt = "Hello"
messages = [{"role": "user", "content": prompt}]
response = openai.ChatCompletion.create(
    model='gpt-4-32k',
    messages=messages,
    temperature=0
    )
    
print( response.choices[0].message["content"] )

InvalidRequestError: The model `gpt-4-32k` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.